In [ ]:
# This notebook looks at the Galileo timing parameters (GAUT and GAGP)
# as derived from the navigation daily rinex files
#
# Octavian Andrei, 2021

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
# import matplotlib
# matplotlib.rcdefaults()

In [ ]:
# helper functions
def parse_line(line):
    # split into time and header strings
    t, hdr = line.values[0][:11], line.values[0][12:]
    
    # time comes from the navigation daily file
    t = pd.to_datetime(t, format='%Y%j%H%M')
    
    # TIME SYSTEM CORR label
    # See Table 5, A21, Rinex v3.04
    typ = hdr[:4]
    a0 = float(hdr[5:22].upper().replace('D','E'))
    a1 = float(hdr[22:38].upper().replace('D','E'))
    T = int(hdr[39:45])
    W = int(hdr[46:50])
    S = hdr[51:56].strip()
    U = int(hdr[57:59])
    
    return pd.Series([typ, t, a0, a1, T, W, S, U ])

def to_gpst(line: pd.Series):
    """Valid only if type is 'GPUT', 'GAUT', 'QZUT', 'IRUT', 'SBUT'
    """
    gw,sow = line[['W', 'T']]
    sec = gw * 7 * 86400 + sow
    ret = pd.Timestamp('1980-01-06') + pd.Timedelta(sec, unit='sec')
    
    return ret

cols = ['TYPE', 'FTIME', 'A0', 'A1', 'T', 'W', 'S', 'U']

In [ ]:
# read input data into a dataframe
fin = '../data/gal_time.gz'
file_content = pd.read_csv(fin, header=None)
df = file_content.apply(parse_line, axis=1)
df = df.rename(columns={i:elem for i, elem in enumerate(cols)})
df['REF_EPOCH'] = df.apply(to_gpst, axis=1)
df

# GAL UTC Time Determination

In [ ]:
has_gaut = df['TYPE'] == 'GAUT'
is_larger_than_a_week = df['T'] > 86400*7
is_after_20161201 = df['REF_EPOCH'] >= pd.Timestamp('2016-11-30')

mask = has_gaut & ~is_larger_than_a_week & is_after_20161201
GAUT = df[mask].copy()

In [ ]:
GAUT['A0'].describe()

In [ ]:
# SDD, Table 18
is_within_mpl = abs(GAUT['A0']) <= 31e-9
GAUT.loc[:,'MPL'] = is_within_mpl

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))

GAUT[GAUT.MPL==True].plot(ax=ax, x='REF_EPOCH', y='A0', color='green', label='<=31ns', marker='+', ls='None')
GAUT[GAUT.MPL==False].plot(ax=ax, x='REF_EPOCH', y='A0', color='red', label='>31ns', marker='.', ls='None')

ax.set_ylabel('A0 (sec)');

# Galileo - GPS Time Offset (GGTO)

In [ ]:
# select and clean the data
has_ggot = df['TYPE'] == 'GPGA'
is_larger_than_a_week = df['T'] > 86400*7
is_after_20161201 = df['REF_EPOCH'] >= pd.Timestamp('2016-11-30')

mask = has_ggot & ~is_larger_than_a_week & is_after_20161201
GGOT = df[mask].copy()

In [ ]:
# SDD, Table 19
is_within_mpl = abs(GGOT['A0']) <= 20e-9
GGOT.loc[:,'MPL'] = is_within_mpl

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))

GGOT[GGOT.MPL==True].plot(ax=ax, x='REF_EPOCH', y='A0', color='green', label='<=20ns', marker='+', ls='None')
GGOT[GGOT.MPL==False].plot(ax=ax, x='REF_EPOCH', y='A0', color='red', label='>20ns', marker='.', ls='None')

ax.set_ylabel('A0 (sec)');